In [1]:
from mpi4py import MPI
from dolfinx import mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


In [2]:
from dolfinx.fem import FunctionSpace
V = FunctionSpace(domain, ("CG", 1))

INFO:root:running build_ext
INFO:root:building 'libffcx_elements_744778381fa4bc2bf426afe9688c3c5a6e57eec1' extension
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/lib/python3.10/site-packages/ffcx/codegeneration -I/opt/conda/include/python3.10 -c libffcx_elements_744778381fa4bc2bf426afe9688c3c5a6e57eec1.c -o ./libffcx_elements_744778381fa4bc2bf426afe9688c3c5a6e57eec1.o -O2 -g0
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/c

In [3]:
from dolfinx import fem
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

In [4]:
import numpy
# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

In [5]:
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

In [6]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [7]:
from petsc4py.PETSc import ScalarType
f = fem.Constant(domain, ScalarType(-6))

In [8]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

In [9]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_2c89199f69731a52ae9a693eec30521ad9f510ab' extension
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/lib/python3.10/site-packages/ffcx/codegeneration -I/opt/conda/include/python3.10 -c libffcx_forms_2c89199f69731a52ae9a693eec30521ad9f510ab.c -o ./libffcx_forms_2c89199f69731a52ae9a693eec30521ad9f510ab.o -O2 -g0
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib 

In [10]:
V2 = fem.FunctionSpace(domain, ("CG", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

INFO:root:running build_ext
INFO:root:building 'libffcx_elements_e19fc72bcf41f0f60850516c5976b4290cd17c95' extension
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/lib/python3.10/site-packages/ffcx/codegeneration -I/opt/conda/include/python3.10 -c libffcx_elements_e19fc72bcf41f0f60850516c5976b4290cd17c95.c -o ./libffcx_elements_e19fc72bcf41f0f60850516c5976b4290cd17c95.o -O2 -g0
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/c

In [11]:
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_43607ee530d1247b4f5fd6a7cea6dd72c914e6a9' extension
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/lib/python3.10/site-packages/ffcx/codegeneration -I/opt/conda/include/python3.10 -c libffcx_forms_43607ee530d1247b4f5fd6a7cea6dd72c914e6a9.c -o ./libffcx_forms_43607ee530d1247b4f5fd6a7cea6dd72c914e6a9.o -O2 -g0
INFO:root:/opt/conda/bin/x86_64-conda-linux-gnu-cc -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib 

In [12]:
error_max = numpy.max(numpy.abs(uD.x.array-uh.x.array))
# Only print the error on one process
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

Error_L2 : 8.24e-03
Error_max : 4.00e-15


In [13]:
import pyvista
print(pyvista.global_theme.jupyter_backend)

trame


In [14]:
from dolfinx import plot
pyvista.start_xvfb()
topology, cell_types, geometry = plot.create_vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

In [15]:
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")

Widget(value="<iframe src='http://localhost:42403/index.html?ui=P_0x7f0b19c77580_0&reconnect=auto' style='widt…

In [16]:
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)

In [17]:
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value="<iframe src='http://localhost:42403/index.html?ui=P_0x7f0b2acf7e20_1&reconnect=auto' style='widt…

In [18]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()

Widget(value="<iframe src='http://localhost:42403/index.html?ui=P_0x7f0b194f1240_2&reconnect=auto' style='widt…

In [19]:
from dolfinx import io
with io.VTXWriter(domain.comm, "output.bp", [uh]) as vtx:
    vtx.write(0.0)
with io.XDMFFile(domain.comm, "output.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh)